In [1]:
import pandas as pd
import requests
import urllib.request, json 
import re
import io
import time
import openpyxl
from datetime import datetime,date

In [2]:
#save timestamp to text file
with open('timestamp.txt', 'w') as f:
    f.write(datetime.now().strftime("%Y-%b-%d"))

In [3]:
# If you need to install something in jupyter notebokes, you need these commands
# import sys
# !{sys.executable} -m pip install openpyxl

In [4]:
# As we only track prices for 5 years, you need to set that time point as a reference point.
# Also as we are using the last January as a reference point, we need to set that as well.

# set average price reference month
avgpriceRefMonth=pd.Timestamp('2023-01-01 00:00:00')

# starting reference point
startref=pd.Timestamp('2018-01-01 00:00:00')

In [5]:
#read in metadata
meta = pd.read_csv('./metadata.csv',index_col=0,parse_dates=['ITEM_START'],date_format="%Y%m")

In [6]:
# define a function to split a string at a certain occurance of a separator

# https://stackoverflow.com/questions/36300158/split-text-after-the-second-occurrence-of-character
def split(strng, sep, pos):
    strng = strng.split(sep)
    return sep.join(strng[:pos]), sep.join(strng[pos:])

In [7]:
# read in unchained csv
unchained = pd.read_csv('unchained.csv')

#find the last month in the unchained file
# latestmonth=datetime.strptime(unchained.columns[-1],"%Y-%m-%d %H:%M:%S")
latestmonth=datetime.strptime(unchained.columns[-1],"%Y-%m-%d")
# and print it out
latestmonth

datetime.datetime(2024, 6, 1, 0, 0)

In [19]:
# Excel changes date formats into something funky. Use this if you need to convert it to python date format
# un = pd.read_csv('unchained.csv', index_col=0)
# columns = {}
# for col in un.columns:
#     try:
#         columns[col] = datetime.strptime(str(col), "%d/%m/%Y").date()
#         # columns[col] = datetime.strptime(str(col), "%d/%m/%Y %H:%M").date()
#         # columns[col] = datetime.strptime(str(col), "%Y-%m-%d %H:%M:%S").date()
#         # columns[col] = datetime.strptime(str(col), "%Y%m").date()
#     except ValueError:
#         pass
# un.rename(columns=columns, inplace=True)
# un.to_csv('unchained.csv',date_format='%Y-%m-%d')

In [10]:
# first get the /data.json from the cpi items and prices page from the ONS website
with requests.Session() as s:
    r=s.get("https://www.ons.gov.uk/economy/inflationandpriceindices/datasets/consumerpriceindicescpiandretailpricesindexrpiitemindicesandpricequotes/data",headers={'User-Agent': 'Mozilla/5.0'})
    data = r.json()
    datasets = data['datasets']

#go through the dataset and find the first one which doesn't contain the word framework, glossary or /pricequotes. The url includes pricesquotes so that slash is important. Save the index as the variable match  
for i,dataset in enumerate(datasets):
    match = i
    if('framework' not in dataset['uri'] and 'glossary' not in dataset['uri'] and '/pricequotes' not in dataset['uri']):
        break
    
#get the uri of the items dataset we want
items = data['datasets'][match]['uri']
print('dataset='+items)

#get the month and year from the uri
date=split(items,'itemindices',2)[1]
print('the date from url:'+date)

#parse it as a date
itemmonth=datetime.strptime(date,"%B%Y")

# check date to see if you need to download a file
if(itemmonth!=latestmonth):
    print('month from indices is different to latest month in unchained csv')
    
    with requests.Session() as s:
        r=s.get("https://www.ons.gov.uk"+items+"/data",headers={'User-Agent': 'Mozilla/5.0'})
        itemspage = r.json()
        csv = itemspage['downloads'][0]['file']
    
    # get the csv of the latest indices
    with requests.Session() as s:
        download = s.get("https://www.ons.gov.uk/file?uri="+items+"/"+csv,headers={'User-Agent': 'Mozilla/5.0'})
        df=pd.read_csv(io.StringIO(download.content.decode('utf-8')))
    
    #get the index date which is the first cell
    index_date=df.iloc[0,0]
    
    # parse columns as dates in unchained
    # https://stackoverflow.com/questions/42472418/parse-file-headers-as-date-objects-in-python-pandas
    columns = {}
    for col in unchained.columns:
        try:
            columns[col] = datetime.strptime(str(col), "%Y-%m-%d")
        except ValueError:
            pass
    unchained.rename(columns=columns, inplace=True)
    
    #join it onto existing csv
    un=unchained.merge(df[['ITEM_ID','ALL_GM_INDEX']].rename(columns={"ALL_GM_INDEX": datetime.strptime(str(index_date),"%Y%m")}),on='ITEM_ID',how='left')
    
    #if last date is Jan, then chain it to december
    if(un.columns[-1].month == 1):
        print('chaining jan')
        jancol=un.columns[-1]
        prevdec=un.columns[-2]
        for index,value in un.iloc[:,-1].items():
            un.at[index,jancol]=un.loc[index,prevdec]*value/100
    
    un.set_index("ITEM_ID",inplace=True)

else:
    print('Nothing to update')  
    # parse columns as dates in unchained
    # https://stackoverflow.com/questions/42472418/parse-file-headers-as-date-objects-in-python-pandas
    columns = {}
    for col in unchained.columns:
        try:
            columns[col] = datetime.strptime(str(col), "%Y-%m-%d")
            # columns[col] = datetime.strptime(str(col), "%Y-%m-%d %H:%M:%S")
        except ValueError:
            pass
    unchained.rename(columns=columns, inplace=True)
    un=unchained
    un.set_index("ITEM_ID",inplace=True)

dataset=/economy/inflationandpriceindices/datasets/consumerpriceindicescpiandretailpricesindexrpiitemindicesandpricequotes/itemindicesjuly2024
the date from url:july2024
month from indices is different to latest month in unchained csv


In [ ]:
#create a copy of unchained to create the chained indices
chained = un.copy()

for col in chained:
    for i, row_value in chained[col].items():
        # print(col,i,row_value,meta.loc[i,'ITEM_START'])
        if(col>=meta.loc[i,'ITEM_START']):
            if(col==startref):
                chained.at[i,col]=100
            # elif(col==meta.loc[i,'ITEM_START']):
            #     sample.at[i,col]=row_value
            elif(col<=startref+pd.tseries.offsets.DateOffset(years=1)):
                chained.at[i,col]=row_value
            else:
                if(col.month==1 and col>startref+pd.tseries.offsets.DateOffset(years=1)):
                    chained.at[i,col]=float(row_value)*float(chained.loc[i][datetime(col.year-1,1,1)])/100
                else:
                    chained.at[i,col]=float(row_value)*float(chained.loc[i][datetime(col.year,1,1)])/100

        elif(col==meta.loc[i,'ITEM_START']-pd.tseries.offsets.DateOffset(months=1)):
            chained.at[i,col]=100

        else:
            chained.at[i,col]=None

In [ ]:
# Then calculate average prices
avgprice=chained.copy()

for col in avgprice:
    for i, row_value in avgprice[col].items():
        if(row_value==None):
            avgprice.at[i,col]=None
        else:
            avgprice.at[i,col]=float(row_value)/ \
            float(chained.loc[i,avgpriceRefMonth])* \
            float(meta.loc[i,'AVERAGE_PRICE'])
            
#rename columns to dates without time formats
columns = {}
for col in avgprice.columns:
    try:
        columns[col] = col.date()
    except ValueError:
        pass
avgprice.rename(columns=columns, inplace=True)

avgprice.astype(float).round(2).to_csv('avgprice.csv',date_format='%Y-%m-%d',na_rep='')

In [ ]:
# calculate annual growth
annualgrowth=chained.copy()

for col in annualgrowth:
    for i, row_value in annualgrowth[col].items():
        if(col<meta.loc[i,'ITEM_START']+pd.tseries.offsets.DateOffset(years=1,months=-1)):
            annualgrowth.at[i,col]=None
        else:
            if(col<startref+pd.tseries.offsets.DateOffset(years=1)):
                annualgrowth.at[i,col]=None
            else:
                annualgrowth.at[i,col]=(float(row_value)- \
                float(chained.loc[i,col-pd.tseries.offsets.DateOffset(years=1)])) * 100 / \
                float(chained.loc[i,col-pd.tseries.offsets.DateOffset(years=1)]) 
                
                
#rename columns to dates without time formats
columns = {}
for col in annualgrowth.columns:
    try:
        columns[col] = col.date()
    except ValueError:
        pass
annualgrowth.rename(columns=columns, inplace=True)
                
annualgrowth.astype(float).round(0).astype(int,errors='ignore').to_csv('annualgrowth.csv',date_format='%Y-%m-%d',na_rep='',float_format="%.0f")

In [14]:
chained

,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,...,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01
ITEM_ID,,,,,,,,,,,,,,,,,,,,,
520130,106.2,100.5,100.4,100.8,101.1,101.5,101.9,101.9,101.8,101.7,...,103.171,103.512,103.590,104.303,104.628425,100.668,101.100,101.660,102.598,102.651
520131,102.9,103.6,102.7,102.1,102.3,100.2,99.8,101.3,101.6,101.6,...,104.996,105.725,105.627,104.516,103.183421,101.055,101.604,100.958,101.136,99.470
520137,108.3,99.6,99.5,98.9,99.6,102.2,101.8,103.1,101.6,100,...,106.581,105.616,104.579,105.486,103.811937,100.924,101.505,100.543,103.076,103.302
520140,103.4,103.1,104.9,104.6,106.6,103,101.5,103.9,106.5,104.1,...,106.510,107.907,108.389,109.608,106.663929,100.124,101.761,101.934,101.117,101.672
510439,-,-,-,-,-,-,-,-,-,-,...,107.534,106.664,106.459,108.471,107.889595,100.920,101.053,100.776,100.803,100.781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620315,104.6,100.5,100.3,100.5,100.6,101,101.5,102.4,103.5,103.9,...,103.140,103.525,104.161,105.209,105.416262,100.102,100.121,100.822,102.328,102.896
610204,104.7,100.5,100.9,101.1,101.6,101.7,101.6,102.1,102.2,101.7,...,104.438,103.350,102.832,101.731,102.009743,100.118,100.593,100.628,101.437,102.328
610303,105.3,100.6,100.4,97.8,97.2,101.2,101.1,98,102.2,100.8,...,100.837,102.039,99.620,99.211,101.679370,100.583,98.466,97.569,100.088,100.241


In [ ]:
# calculate monthly growth
monthlygrowth=chained.copy()

for col in monthlygrowth:
    for i, row_value in monthlygrowth[col].items():
        if(col<meta.loc[i,'ITEM_START']):
            monthlygrowth.at[i,col]=None
        else:
            if(col<startref+pd.tseries.offsets.DateOffset(months=1)):
                monthlygrowth.at[i,col]=None
            else:
                monthlygrowth.at[i,col]=(float(row_value)- \
                float(chained.loc[i,col-pd.tseries.offsets.DateOffset(months=1)])) * 100 / \
                float(chained.loc[i,col-pd.tseries.offsets.DateOffset(months=1)])

#rename columns to dates without time formats
columns = {}
for col in monthlygrowth.columns:
    try:
        columns[col] = col.date()
    except ValueError:
        pass
monthlygrowth.rename(columns=columns, inplace=True)
                
monthlygrowth.astype(float).round(0).astype(int,errors='ignore').to_csv('monthlygrowth.csv',date_format='%Y-%m-%d',na_rep='',float_format="%.0f")

In [ ]:
#Finally save the unchained and chainedvnumbers to csv
#rename columns to dates without time formats
columns = {}
for col in un.columns:
    try:
        columns[col] = col.date()
    except ValueError:
        pass
un.rename(columns=columns, inplace=True)

#and save it
un.to_csv('unchained.csv')

#rename columns to dates without time formats
columns = {}
for col in chained.columns:
    try:
        columns[col] = col.date()
    except ValueError:
        pass
chained.rename(columns=columns, inplace=True)

chained.astype(float).round(3).to_csv('chained.csv',date_format='%Y-%m-%d',na_rep='')

In [17]:
#turn it into a excel datadownload file
# with pd.ExcelWriter("datadownload.xlsx", mode="a", if_sheet_exists="replace", date_format="YYYY-MM-DD", datetime_format="YYYY-MM-DD") as writer:
#     meta.drop(columns=['AVERAGE_PRICE']).to_excel(writer, sheet_name="metadata")  
#     # un.to_excel(writer, sheet_name="unchained")
#     chained.astype(float).round(3).transpose().to_excel(writer, sheet_name="chained")
#     avgprice.astype(float).round(2).fillna('').transpose().to_excel(writer, sheet_name="averageprice")
#     monthlygrowth.astype(float).round(0).fillna('').transpose().to_excel(writer, sheet_name="monthlygrowth")
#     annualgrowth.astype(float).round(0).fillna('').transpose().to_excel(writer,sheet_name="annualgrowth")
    

In [ ]:
#turn it into a excel datadownload file
with pd.ExcelWriter("datadownload.xlsx", mode="a", if_sheet_exists="replace", date_format="YYYY-MM-DD", datetime_format="YYYY-MM-DD") as writer:
    meta.drop(columns=['AVERAGE_PRICE']).to_excel(writer, sheet_name="Metadata")  
    # un.to_excel(writer, sheet_name="unchained")
    
    # make it tidy, join on meta data, reorder columns by index
    chained.astype(float).round(3).reset_index().melt(id_vars=['ITEM_ID'],var_name='Date',value_name='Value').dropna()\
    .merge(meta.reset_index()[['Category1','Category2','ITEM_ID','ITEM_DESC','WEIGHT\SIZE']])\
    .iloc[:,[1,0,3,4,5,6,2]]\
    .to_excel(writer, index=False, sheet_name="Chained")
    
    avgprice.astype(float).round(2).fillna('').reset_index().melt(id_vars=['ITEM_ID'],var_name='Date',value_name='Price').dropna()\
    .merge(meta.reset_index()[['Category1','Category2','ITEM_ID','ITEM_DESC','WEIGHT\SIZE']])\
    .iloc[:,[1,0,3,4,5,6,2]]\
    .to_excel(writer, index=False, sheet_name="Average price")
    
    monthlygrowth.astype(float).round(0).fillna('').reset_index().melt(id_vars=['ITEM_ID'],var_name='Date',value_name='Percentage').dropna() \
    .merge(meta.reset_index()[['Category1','Category2','ITEM_ID','ITEM_DESC','WEIGHT\SIZE']])\
    .iloc[:,[1,0,3,4,5,6,2]]\
    .to_excel(writer, index=False, sheet_name="Monthly growth")
    
    annualgrowth.astype(float).round(0).fillna('').reset_index().melt(id_vars=['ITEM_ID'],var_name='Date', value_name='Percentage').dropna() \
    .merge(meta.reset_index()[['Category1','Category2','ITEM_ID','ITEM_DESC','WEIGHT\SIZE']])\
    .iloc[:,[1,0,3,4,5,6,2]]\
    .to_excel(writer,index=False, sheet_name="Annualgrowth")
    